In [ ]:
!pip install pydicom opencv-python-headless[app] nibabel matplotlib albumentations tqdm

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import os
import glob
import cv2
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from albumentations import HorizontalFlip, VerticalFlip, Rotate
from tqdm import tqdm
import torch.nn.functional as F
import matplotlib.image as mpimg
import albumentations as A
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import jaccard_score
from tqdm import tqdm
import nibabel as nib
import albumentations as A
from PIL import Image
from torchvision.transforms import ToTensor, Normalize

height,width = (256,256) 


torch.cuda.is_available()

c:\Users\Zeynep Aygün\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Zeynep Aygün\AppData\Roaming\Python\Python39\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")


True

In [2]:
import torch
import numpy as np
from torch.utils.data import Dataset
import nibabel as nib
import albumentations as A
import pydicom
import cv2


class LoadData_imonly(Dataset):
    def __init__(self, images_path, classes):
        super().__init__()

        self.images_path = images_path
        self.classes = classes
        self.len = len(images_path)
        self.transform = A.Compose([
            A.Resize(height=height,width=width,p=1.0),
            A.augmentations.transforms.CLAHE(clip_limit=(2.0,3.0), tile_grid_size=(8, 8), always_apply=False, p=0.5) 
        ])
        self.to_tensor = ToTensor()
        self.normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    def __getitem__(self, idx):
        img = pydicom.dcmread(self.images_path[idx]).pixel_array
        img=np.invert(img)
        img = ((img - img.min()) * (1/(img.max() - img.min()) * 255)).astype('uint8')
        
        transformed = self.transform(image=img)
        img = transformed["image"]
        img = np.array([img,img,img])

        img=np.moveaxis(img,0,-1)
        img = self.to_tensor(img)
        return img

    def __len__(self):
        return self.len


In [3]:
from torch.utils.data import DataLoader, random_split

# Veri kümesini oluşturun
# PATH="/media/uraninjo/7D72-19E7/teknofest/"

PATH="C:/Users/Zeynep Aygün/Desktop/ornek"

test_images_path=sorted(glob.glob(f'{PATH}/images/[Ertan,Zeynep]*/*/*.dcm'))  # Görüntü yollarının listesi

classes = [0, 1, 2]  # Orijinal etiket değerleri
print(len(test_images_path))

360


In [4]:
test_dataset = LoadData_imonly(test_images_path, classes)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [ ]:
imgs=test_dataset[4]

plt.imshow(np.transpose(imgs,(1,2,0)),cmap="inferno")
plt.imshow

In [6]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.double_conv(x)
    
    
class DownBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DownBlock, self).__init__()
        self.double_conv = DoubleConv(in_channels, out_channels)
        self.down_sample = nn.MaxPool2d(2)

    def forward(self, x):
        skip_out = self.double_conv(x)
        down_out = self.down_sample(skip_out)
        return (down_out, skip_out)

    
class UpBlock(nn.Module):
    def __init__(self, in_channels, out_channels, up_sample_mode):
        super(UpBlock, self).__init__()
        if up_sample_mode == 'conv_transpose':
            self.up_sample = nn.ConvTranspose2d(in_channels-out_channels, in_channels-out_channels, kernel_size=2, stride=2)        
        elif up_sample_mode == 'bilinear':
            self.up_sample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            raise ValueError("Unsupported `up_sample_mode` (can take one of `conv_transpose` or `bilinear`)")
        self.double_conv = DoubleConv(in_channels, out_channels)

    def forward(self, down_input, skip_input):
        x = self.up_sample(down_input)
        x = torch.cat([x, skip_input], dim=1)
        return self.double_conv(x)

    
class UNet(nn.Module):
    def __init__(self, out_classes=3, up_sample_mode='conv_transpose'):
        super(UNet, self).__init__()
        self.up_sample_mode = up_sample_mode
        # Downsampling Path
        self.down_conv1 = DownBlock(3, 64)
        self.down_conv2 = DownBlock(64, 128)
        self.down_conv3 = DownBlock(128, 256)
        self.down_conv4 = DownBlock(256, 512)
        # Bottleneck
        self.double_conv = DoubleConv(512, 1024)
        # Upsampling Path
        self.up_conv4 = UpBlock(512 + 1024, 512, self.up_sample_mode)
        self.up_conv3 = UpBlock(256 + 512, 256, self.up_sample_mode)
        self.up_conv2 = UpBlock(128 + 256, 128, self.up_sample_mode)
        self.up_conv1 = UpBlock(128 + 64, 64, self.up_sample_mode)
        # Final Convolution
        self.conv_last = nn.Conv2d(64, out_classes, kernel_size=1)
        
        

    def forward(self, x):
        x, skip1_out = self.down_conv1(x)
        x, skip2_out = self.down_conv2(x)
        x, skip3_out = self.down_conv3(x)
        x, skip4_out = self.down_conv4(x)
        x = self.double_conv(x)
        x = self.up_conv4(x, skip4_out)
        x = self.up_conv3(x, skip3_out)
        x = self.up_conv2(x, skip2_out)
        x = self.up_conv1(x, skip1_out)
        x = self.conv_last(x)
        
        return x
    

# Get UNet model
model = UNet()
model

UNet(
  (down_conv1): DownBlock(
    (double_conv): DoubleConv(
      (double_conv): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (down_sample): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (down_conv2): DownBlock(
    (double_conv): DoubleConv(
      (double_conv): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      

In [7]:
import pydicom
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
class Temp:
    _temp_img=None
    _temp_mock=None
    rel_vals=None

def crop_and_mask_dcm_image(masks, idx):
    real_img = pydicom.dcmread(test_images_path[idx]).pixel_array
    real_img=np.invert(real_img)
    real_img = ((real_img - real_img.min()) * (1/(real_img.max() - real_img.min()) * 255)).astype('uint8')
    real_w,real_h=real_img.shape
    mock_masks=cv2.resize(masks,(real_h,real_w), interpolation=cv2.INTER_AREA)
    
    Temp._temp_mock=mock_masks
    rslt = mock_masks* real_img

    rows, cols = np.where(mock_masks!= 0)
    y1, x1 = np.min(rows), np.min(cols)
    y2, x2 = np.max(rows), np.max(cols)
    # y1_rel,y2_rel=y1/256,y2/256
    # x1_rel,x2_rel=x1/256,x2/256
    # Temp.rel_vals=(y1_rel,y2_rel,x1_rel,x2_rel)
    # print(y1,y2)
    # print(x1,x2)


    # Görüntüyü bounding box ile kırp
    result = rslt[y1:y2+1, x1:x2+1]

    # Maske dışındaki alanı siyah renge dök
    result[mock_masks[y1:y2+1, x1:x2+1] == 0] = 0
    Temp._temp_img=result

    return imgs, rslt, result

In [12]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import DataLoader

def visualize_test_mask(output_np,masks):
    fig, axs = plt.subplots(1, 2)
    img=np.transpose(output_np,(1,2,0))
    axs[0].imshow(img,cmap="gray")
    axs[0].set_title("Predicted mask")
    masks=np.transpose(masks.cpu(),(1,2,0))
    axs[1].imshow(masks)
    axs[1].set_title("Ground truth mask")
    plt.show()

def run_unet_test(model_path, test_dataset, device):
    try:
        os.mkdir(f"C:/Users/Zeynep Aygün/Desktop/teknofest_github/Mergen1-Teknofest/zeynep/çalışmalar_zeynep")
    except:
        print("Result folder already exists")
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)
    model = UNet()
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()
    with torch.no_grad():
        for i, inputs in enumerate(test_loader):
            inputs = inputs.to(device)
            outputs = model(inputs)
            output_np = outputs.detach().cpu().numpy()
            output_np = output_np - np.min(output_np)
            output_np = output_np / np.max(output_np)
            predicted_masks = np.around(output_np, decimals=0, out=None)

            mask_1 = predicted_masks[0][1,:,:]
            mask_2 = predicted_masks[0][2,:,:]
            combined_mask = np.maximum(mask_1, mask_2)
            
            imgs, rslt, result = crop_and_mask_dcm_image(combined_mask,i)#inputs[0][0,:,:].cpu().numpy()
            fname=test_images_path[i]
            __tür=fname.split("/")[-1].split(".")[0]
            __hno=fname.split("/")[-2]
            
            try:
                os.mkdir(f"C:/Users/Zeynep Aygün/Desktop/teknofest_github/Mergen1-Teknofest/zeynep/çalışmalar_zeynep/{__hno}")
            except:
                print("Folder already exists")
            plt.imsave(f"C:/Users/Zeynep Aygün/Desktop/teknofest_github/Mergen1-Teknofest/zeynep/çalışmalar_zeynep/{__hno}/{__tür}.png",result,cmap="gray")

            if i==20:
                break
            # plt.imshow(result,cmap="gray")
            # visualize_test_mask(predicted_masks[0], inputs[0])

run_unet_test("C:/Users/Zeynep Aygün/Desktop/teknofest_github/Mergen1-Teknofest/zeynep/checkpoints_2/6_ValLoss0.5481_diceScore0.9260.pt", test_dataset, "cuda")


Result folder already exists
Folder already exists


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Zeynep Aygün/Desktop/teknofest_github/Mergen1-Teknofest/zeynep/çalışmalar_zeynep/ornek/images\\Ertan\\822670817\\LCC.png'